<a href="https://colab.research.google.com/github/imabari/rakuten_tools/blob/main/rakuten_towercollector_near.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 実行方法

1. 上の「Open in Colab」をクリック（Googleアカウント必要）
2. 上部メニューより「ランタイム」を選択
3. 「すべてのセルを実行」を選択
4. アップロードのところで停止するので参照を押してTowerCollectorのファイルを指定

# アップロード

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

In [ ]:
!pip install geopandas

# プログラム

In [3]:
import pandas as pd

In [ ]:
df0 = (
    pd.read_csv(fn, parse_dates=["measured_at"])
    .dropna(how="all", axis=1)
    .dropna(how="any", subset=["cell_id", "psc", "ta"])
    .astype({"cell_id": int, "short_cell_id": int, "rnc": int, "ta": int})
)
df0

In [5]:
# 型確認
df0.dtypes

mcc                            int64
mnc                            int64
lac                            int64
cell_id                        int64
short_cell_id                  int64
rnc                            int64
psc                            int64
asu                            int64
dbm                            int64
ta                             int64
lat                          float64
lon                          float64
accuracy                     float64
speed                        float64
bearing                      float64
altitude                     float64
measured_at      datetime64[ns, UTC]
net_type                      object
neighboring                     bool
device                        object
rsrp                           int64
rsrq                           int64
rssi                           int64
rssnr                        float64
arfcn                          int64
dtype: object

In [ ]:
# バンド指定
df1 = df0.query("mcc == 440 & mnc == 11").copy()
df1

In [8]:
df1["id"] = df1["short_cell_id"].astype(str) + "-" + df1["rnc"].astype(str)

# PCI確認

In [9]:
df_pci = (
    df1.drop_duplicates(subset=["cell_id"])
    .pivot(index="short_cell_id", columns="rnc", values="psc")
    .astype("Int64")
)
df_pci.fillna(0).astype(str).replace("0", "-")

rnc,1,2,3,4,5,6,7,8,9,10,11,12
short_cell_id,,,,,,,,,,,,
737354,298,89,174,225,-,205,-,-,-,116,229,348
737359,82,-,263,-,-,-,-,-,-,-,-,-
737363,-,-,-,-,-,-,-,-,-,345,-,-
737366,-,-,-,185,183,-,51,133,137,-,-,-
737378,194,-,130,-,147,169,-,317,213,-,-,96
737403,-,-,-,-,-,-,-,-,242,-,-,-
737422,-,-,-,247,38,270,-,-,-,-,-,-
737429,-,-,-,125,-,-,-,-,-,-,-,-
737486,-,-,-,-,-,-,263,261,262,75,175,77


In [10]:
df_pci.to_csv("pci.csv", encoding="utf_8_sig")

# 最良値を抽出

In [11]:
df2 = df1.groupby(["cell_id"])["dbm"].max().reset_index()
df2

,cell_id,dbm
0,188762625,-51
1,188762626,-51
2,188762627,-51
3,188762628,-51
4,188762630,-51
5,188762634,-51
6,188762635,-51
7,188762636,-51
8,188763905,-57
9,188763907,-53


In [ ]:
df3 = pd.merge(df1, df2)
df3

In [13]:
import geopandas as gpd
from shapely.geometry import Point

In [14]:
# 全部
# geo_df = gpd.GeoDataFrame(df1, geometry=gpd.points_from_xy(df1.lon, df1.lat), crs=2446)

In [15]:
# 最良値のみ
geo_df = gpd.GeoDataFrame(df3, geometry=gpd.points_from_xy(df3.lon, df3.lat), crs=2446)

In [ ]:
geo_df

In [17]:
# スプレッドシートのURL
csv_url = "https://raku10ehime.github.io/map/ehime.csv"

In [18]:
df_ehime = pd.read_csv(csv_url, index_col=0)
df_ehime

,場所,市区町村,場所補足,設置タイプ,状況,sector,sub6,ミリ波,eNB-LCID,PCI,基地局ID,緯度,経度,更新日時,color,icon
0,【5G】今治市立花町二丁目,今治市,NaN,コン柱,open,3.0,3.0,3.0,"737335-1,2,3","39,347,139",NaN,34.052319,132.999271,2022/06/01 0:00:00,darkblue,plane
1,【5G】今治市中寺①,今治市,今治中寺郵便局付近,コン柱,open,3.0,3.0,3.0,"737331-1,2,3","292,98,300",NaN,34.037485,132.991341,2022/06/01 0:00:00,darkblue,plane
2,今治市桜井四丁目,今治市,志島ヶ原・綱敷天満神社付近,コン柱,open,3.0,NaN,NaN,"737512-10,11,12","47,378,94",NaN,34.017394,133.041464,2023/04/23 16:44:15,green,signal
3,今治市登畑,今治市,レデイ薬局今治国分店付近,コン柱,open,3.0,NaN,NaN,"737510-4,5,6","136,278,135",NaN,34.019522,133.016295,2023/04/23 18:22:03,green,signal
4,【5G】今治市阿方①,今治市,ファミリーマート今治阿方店付近,コン柱,open,3.0,1.0,2.0,"737354-4,5,6","225,227,226",NaN,34.061198,132.970841,2022/11/13 2:31:56,darkblue,plane
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,西予市明浜町狩浜,西予市,NaN,コン柱,open,3.0,NaN,NaN,"737523-4,5,6","117,373,374",NaN,33.318979,132.467736,2023/04/22 16:49:28,green,signal
697,西予市明浜町高山,西予市,NaN,コン柱,open,3.0,NaN,NaN,"737523-7,8,9","38,181,36",CBSB838000416,33.321618,132.433275,2023/04/22 16:59:01,green,signal
698,西予市三瓶町垣生,西予市,NaN,コン柱,open,1.0,NaN,NaN,737575-5,10,NaN,33.386473,132.409811,2023/04/22 17:28:16,green,signal
699,今治市小浦町,今治市,サンライズ糸山,コン柱,open,3.0,NaN,NaN,"737378-10,11,12","14,145,96",NaN,34.109348,132.978349,2023/04/23 16:08:12,green,signal


In [19]:
dfs = []

for i, r in df_ehime.iterrows():
    
    point = Point(r["経度"], r["緯度"])

    # 300mの範囲
    buffer = point.buffer(0.003)

    df_tmp = geo_df[geo_df.geometry.within(buffer)].copy()

    if len(df_tmp) > 0:

        df_tmp["grp"] = i

        df_tmp["場所"] = r["場所"]

        df_tmp["距離"] = df_tmp.geometry.apply(lambda x: round(x.distance(point) * 10000, 3))

        dfs.append(df_tmp)


In [20]:
df4 = pd.concat(dfs).sort_values(by=["cell_id", "距離"]).reset_index(drop=True)
df4

,mcc,mnc,lac,cell_id,short_cell_id,rnc,psc,asu,dbm,ta,...,rsrp,rsrq,rssi,rssnr,arfcn,id,geometry,grp,場所,距離
0,440,11,39937,188762625,737354,1,298,31,-51,0,...,-65,-8,-51,11.0,1500,737354-1,POINT (132.966 34.063),5,【5G】今治市阿方②,4.892
1,440,11,39937,188762625,737354,1,298,31,-51,0,...,-73,-10,-51,11.0,1500,737354-1,POINT (132.967 34.062),5,【5G】今治市阿方②,13.359
2,440,11,39937,188762625,737354,1,298,31,-51,0,...,-73,-10,-51,11.0,1500,737354-1,POINT (132.967 34.062),5,【5G】今治市阿方②,18.788
3,440,11,39937,188762626,737354,2,89,31,-51,0,...,-61,-11,-51,12.0,1500,737354-2,POINT (132.965 34.062),5,【5G】今治市阿方②,2.874
4,440,11,39937,188762627,737354,3,174,31,-51,0,...,-46,-8,-51,11.0,1500,737354-3,POINT (132.965 34.063),5,【5G】今治市阿方②,3.560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,440,11,39937,188796938,737488,10,109,31,-51,0,...,-76,-10,-51,11.0,1500,737488-10,POINT (132.953 34.119),51,今治市波方町波方,6.798
251,440,11,39937,188796938,737488,10,109,31,-51,1,...,-62,-9,-51,15.0,1500,737488-10,POINT (132.953 34.117),51,今治市波方町波方,19.106
252,440,11,39937,188796938,737488,10,109,31,-51,1,...,-62,-9,-51,15.0,1500,737488-10,POINT (132.953 34.117),51,今治市波方町波方,22.257
253,440,11,39937,188796938,737488,10,109,31,-51,1,...,-62,-9,-51,15.0,1500,737488-10,POINT (132.953 34.116),51,今治市波方町波方,26.688


In [21]:
# 重複除去
df5 = df4.drop_duplicates(subset="cell_id").copy()

In [22]:
# 2つ以上を抽出、重複除去
# df5 = df4.groupby(by="cell_id").filter(lambda x: len(x) > 1).drop_duplicates(subset="cell_id").copy()

In [23]:
df5

,mcc,mnc,lac,cell_id,short_cell_id,rnc,psc,asu,dbm,ta,...,rsrp,rsrq,rssi,rssnr,arfcn,id,geometry,grp,場所,距離
0,440,11,39937,188762625,737354,1,298,31,-51,0,...,-65,-8,-51,11.0,1500,737354-1,POINT (132.966 34.063),5,【5G】今治市阿方②,4.892
3,440,11,39937,188762626,737354,2,89,31,-51,0,...,-61,-11,-51,12.0,1500,737354-2,POINT (132.965 34.062),5,【5G】今治市阿方②,2.874
4,440,11,39937,188762627,737354,3,174,31,-51,0,...,-46,-8,-51,11.0,1500,737354-3,POINT (132.965 34.063),5,【5G】今治市阿方②,3.560
11,440,11,39937,188762628,737354,4,225,31,-51,0,...,-50,-9,-51,21.0,1500,737354-4,POINT (132.971 34.061),4,【5G】今治市阿方①,1.252
27,440,11,39937,188762630,737354,6,205,31,-51,0,...,-54,-7,-51,17.0,1500,737354-6,POINT (132.971 34.061),4,【5G】今治市阿方①,1.105
35,440,11,39937,188762634,737354,10,116,31,-51,0,...,-58,-8,-51,22.0,1500,737354-10,POINT (132.956 34.066),16,【5G】今治市延喜①,6.338
42,440,11,39937,188762635,737354,11,229,31,-51,0,...,-56,-8,-51,18.0,1500,737354-11,POINT (132.955 34.065),16,【5G】今治市延喜①,0.489
48,440,11,39937,188762636,737354,12,348,31,-51,0,...,-60,-10,-51,22.0,1500,737354-12,POINT (132.955 34.066),16,【5G】今治市延喜①,0.678
51,440,11,39937,188763905,737359,1,82,28,-57,0,...,-88,-11,-57,8.0,1500,737359-1,POINT (132.988 34.064),22,【5G】今治市北日吉町一丁目,12.432
55,440,11,39937,188765700,737366,4,185,31,-51,0,...,-53,-10,-51,20.0,1500,737366-4,POINT (132.959 34.071),33,今治市延喜②,1.587


In [24]:
df6 = df5.groupby(["grp", "場所", "short_cell_id"])[["rnc", "psc"]].agg(lambda x: list(x)).reset_index()

In [25]:
df6["count"] = df6["rnc"].apply(len)

In [26]:
df6["rnc"] = df6["rnc"].apply(lambda x: ",".join(map(str, x)))
df6["psc"] = df6["psc"].apply(lambda x: ",".join(map(str, x)))

In [27]:
df6["eNB-LCID"] = df6["short_cell_id"].astype(str).str.cat(df6["rnc"], sep="-")

In [35]:
df7 = df6.reindex(columns=["場所", "eNB-LCID", "psc"])
df7

,場所,eNB-LCID,psc
0,【5G】今治市阿方①,"737354-4,6","225,205"
1,【5G】今治市阿方②,"737354-1,2,3","298,89,174"
2,今治市波止浜,"737487-3,4,5","179,178,177"
3,今治市近見町三丁目,"737378-5,6","147,169"
4,【5G】今治市延喜①,"737354-10,11,12","116,229,348"
5,【5G】今治市北日吉町一丁目,737359-1,82
6,今治市延喜②,"737366-4,5","185,183"
7,今治市波方町樋口①,"737488-4,5,6","313,324,326"
8,今治市波方町樋口②,"737486-7,8,9","263,261,262"
9,今治市波方町小部,"737422-4,5,6","247,38,270"


In [36]:
df7.to_csv("enb-lcid.csv", encoding="utf_8_sig")

In [38]:
df8 = df6.join(df_ehime, on="grp", lsuffix="_NEW")

In [40]:
df8[df8["eNB-LCID"] != df8["eNB-LCID_NEW"]]

,grp,場所_NEW,short_cell_id,rnc,psc,count,eNB-LCID_NEW,場所,市区町村,場所補足,...,sub6,ミリ波,eNB-LCID,PCI,基地局ID,緯度,経度,更新日時,color,icon
0,4,【5G】今治市阿方①,737354,"4,6","225,205",2,"737354-4,6",【5G】今治市阿方①,今治市,ファミリーマート今治阿方店付近,...,1.0,2.0,"737354-4,5,6","225,227,226",NaN,34.061198,132.970841,2022/11/13 2:31:56,darkblue,plane
3,14,今治市近見町三丁目,737378,"5,6","147,169",2,"737378-5,6",今治市近見町三丁目,今治市,NaN,...,NaN,NaN,"737378-4,5,6","38,37,36",NaN,34.090902,132.981823,2022/06/01 0:00:00,green,signal
5,22,【5G】今治市北日吉町一丁目,737359,1,82,1,737359-1,【5G】今治市北日吉町一丁目,今治市,NaN,...,1.0,3.0,"737359-1,2,3","82,321,83",NaN,34.062963,132.989182,2023/04/23 21:21:13,darkblue,plane
6,33,今治市延喜②,737366,"4,5","185,183",2,"737366-4,5",今治市延喜②,今治市,乗禅寺付近,...,NaN,NaN,"737366-4,5,6","185,183,184",NaN,34.071344,132.959000,2022/06/01 0:00:00,green,signal


In [41]:
# PCI確認
df5.pivot(
    index=["grp", "場所", "short_cell_id"], columns=["rnc"], values="psc"
).sort_index(level=2).fillna(0).astype(int).astype(str).replace("0", "-")

,,rnc,1,2,3,4,5,6,7,8,9,10,11,12
grp,場所,short_cell_id,,,,,,,,,,,,
4,【5G】今治市阿方①,737354,-,-,-,225,-,205,-,-,-,-,-,-
5,【5G】今治市阿方②,737354,298,89,174,-,-,-,-,-,-,-,-,-
16,【5G】今治市延喜①,737354,-,-,-,-,-,-,-,-,-,116,229,348
22,【5G】今治市北日吉町一丁目,737359,82,-,-,-,-,-,-,-,-,-,-,-
33,今治市延喜②,737366,-,-,-,185,183,-,-,-,-,-,-,-
645,今治市しまなみの杜,737366,-,-,-,-,-,-,51,133,137,-,-,-
14,今治市近見町三丁目,737378,-,-,-,-,147,169,-,-,-,-,-,-
41,今治市波方町小部,737422,-,-,-,247,38,270,-,-,-,-,-,-
40,今治市波方町樋口②,737486,-,-,-,-,-,-,263,261,262,-,-,-


In [33]:
# TA確認
df5.pivot(index=["grp", "場所", "short_cell_id"], columns="rnc", values="ta").sort_index(
    level=2
).fillna(999).astype(int).astype(str).replace("999", "-")

,,rnc,1,2,3,4,5,6,7,8,9,10,11,12
grp,場所,short_cell_id,,,,,,,,,,,,
4,【5G】今治市阿方①,737354,-,-,-,0,-,0,-,-,-,-,-,-
5,【5G】今治市阿方②,737354,0,0,0,-,-,-,-,-,-,-,-,-
16,【5G】今治市延喜①,737354,-,-,-,-,-,-,-,-,-,0,0,0
22,【5G】今治市北日吉町一丁目,737359,0,-,-,-,-,-,-,-,-,-,-,-
33,今治市延喜②,737366,-,-,-,0,0,-,-,-,-,-,-,-
645,今治市しまなみの杜,737366,-,-,-,-,-,-,0,0,0,-,-,-
14,今治市近見町三丁目,737378,-,-,-,-,0,0,-,-,-,-,-,-
41,今治市波方町小部,737422,-,-,-,0,0,0,-,-,-,-,-,-
40,今治市波方町樋口②,737486,-,-,-,-,-,-,0,0,1,-,-,-


In [34]:
# 距離確認
df5.pivot(
    index=["grp", "場所", "short_cell_id"], columns=["rnc"], values="距離"
).sort_index(level=2).fillna(0).astype(str).replace("0", "-")

,,rnc,1,2,3,4,5,6,7,8,9,10,11,12
grp,場所,short_cell_id,,,,,,,,,,,,
4,【5G】今治市阿方①,737354,0.0,0.0,0.0,1.252,0.0,1.105,0.0,0.0,0.0,0.0,0.0,0.0
5,【5G】今治市阿方②,737354,4.892,2.874,3.56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,【5G】今治市延喜①,737354,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.338,0.489,0.678
22,【5G】今治市北日吉町一丁目,737359,12.432,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33,今治市延喜②,737366,0.0,0.0,0.0,1.587,1.305,0.0,0.0,0.0,0.0,0.0,0.0,0.0
645,今治市しまなみの杜,737366,0.0,0.0,0.0,0.0,0.0,0.0,3.068,1.519,1.018,0.0,0.0,0.0
14,今治市近見町三丁目,737378,0.0,0.0,0.0,0.0,1.881,1.881,0.0,0.0,0.0,0.0,0.0,0.0
41,今治市波方町小部,737422,0.0,0.0,0.0,2.729,1.313,2.301,0.0,0.0,0.0,0.0,0.0,0.0
40,今治市波方町樋口②,737486,0.0,0.0,0.0,0.0,0.0,0.0,3.159,3.159,4.896,0.0,0.0,0.0
